Load data

In [6]:
import os
import pandas as pd
import glob
from typing import Tuple, List

In [8]:
path_to_data_folder = os.path.join(os.getcwd(), "data")
print(os.path.join(path_to_data_folder, '*.txt'))
for f in glob.glob(os.path.join(path_to_data_folder, '*.txt')):
    variable_name = os.path.splitext(os.path.basename(f))[0]
    globals()[variable_name] = pd.read_csv(f, delimiter='\t')
    globals()[variable_name].columns = ['Time[s]','VGRF-left1', 'VGRF-left2','VGRF-left3','VGRF-left4','VGRF-left5','VGRF-left6','VGRF-left7','VGRF-left8','VGRF-right1', 'VGRF-right2','VGRF-right3','VGRF-right4','VGRF-right5','VGRF-right6','VGRF-right7','VGRF-right8','Total Force-left','Total Force-right']

E:\PD_gait_analysis\data\*.txt


In [11]:
def divide_dataset_to_Co_and_Pt(path_to_data_folder: str) -> Tuple[List[str], List[str]]:
    """

    Parameters
    ----------
    path_to_data_folder: str
        path to data folder

    Returns
    -------
    file_names_Co: List[str]
        list contains names of files containing Co in name
    file_names_Pt: List[str]
        list contains names of files containing Pt in name
    """

    file_names = os.listdir(path_to_data_folder)
    file_names_Co = [i for i in file_names if 'Co' in i]
    file_names_Pt = [i for i in file_names if 'Pt' in i]
    return file_names_Co, file_names_Pt

In [15]:
file_names_Co, file_names_Pt = divide_dataset_to_Co_and_Pt(path_to_data_folder)

<class 'pandas.core.frame.DataFrame'>


Gait Phase Detection


In [ ]:
def remove_spikes(signal: pd.core.frame.DataFrame) -> List[float]:
    """
    Remove spikes within swing phase

    Parameters
    ----------
    signal: pd.core.frame.DataFrame
        data frame contains dynamic center od pressure during gait

    Returns
    -------
    signal_no_spikes: List[float]
        list contains coordinates of center of pressure for left foot"""
    signal_no_spikes = signal
    for index in range(1,len(signal)):
        if((signal.iloc[index-1]==0 and signal.iloc[index]>0 and signal.iloc[index+1]==0)) or ((signal.iloc[index-1]==0 and signal.iloc[index]>0 and signal.iloc[index+1]>0)):
            signal_no_spikes.iloc[index]=0
        else:
            pass
    return signal_no_spikes

In [115]:
def center_of_pressure_detection(file_name: pd.core.frame.DataFrame) -> Tuple[List[float], List[float]]:

    """
    Parameters
    ----------
    file_name: pd.core.frame.DataFrame
        data frame contains time[s], measurements from 16 pressure sensors - 8 left leg and 8 right leg, and total force for left leg and right leg

    Returns
    -------
    y_left: List[float]
        list contains coordinates of center of pressure for left foot
    y_right: List[float]
        list contains coordinates of center of pressure for right foot
    """

    # L1 (-500,-800) R1 (500,-800)
    # L2 (-700,-400) R2 (700,-400)
    # L3 (-300,-400) R3 (300,-400)
    # L4 (-700,0)    R4 (700,0)
    # L5 (-300,0)    R5 (300,0)
    # L6 (-700,400)  R6 (700,400)
    # L7 (-300,400)  R7 (300,400)
    # L8 (-500,800)  R8 (500,800)

    y_coordinates = {'1': -800,'2': -400,'3': -400,'4': 0,'5': 0,'6': 400,'7': 400,'8': 800}
    filename_col_names_left_leg = ['VGRF-left1', 'VGRF-left2','VGRF-left3','VGRF-left4','VGRF-left5','VGRF-left6','VGRF-left7','VGRF-left8']
    filename_col_names_right_leg = ['VGRF-right1', 'VGRF-right2','VGRF-right3','VGRF-right4','VGRF-right5','VGRF-right6','VGRF-right7','VGRF-right8']

    y_coordinates_left = sum(file_name[filename_col_names_left_leg[i]]*y_coordinates[str(i+1)] for i in range(len(filename_col_names_left_leg)))/file_name['Total Force-left']
    y_coordinates_right = sum(file_name[filename_col_names_right_leg[i]]*y_coordinates[str(i+1)] for i in range(len(filename_col_names_right_leg)))/file_name['Total Force-right']

    # file_name['Total Force-left'] can be 0 because during swing phase total pressure equals to 0
    # then all NaN values are replaced with 0
    y_coordinates_left = y_coordinates_left.fillna(0)
    y_coordinates_right = y_coordinates_right.fillna(0)

    y_coordinates_left = remove_spikes(y_coordinates_left)
    y_coordinates_right = remove_spikes(y_coordinates_right)

    return y_coordinates_left, y_coordinates_right